In [1]:
!git clone https://github.com/Microsoft/vcpkg.git
!cd vcpkg
!./bootstrap-vcpkg.sh
!./vcpkg integrate install
!./vcpkg install sentencepiece

fatal: destination path 'vcpkg' already exists and is not an empty directory.
/bin/bash: line 1: ./bootstrap-vcpkg.sh: No such file or directory
/bin/bash: line 1: ./vcpkg: Is a directory
/bin/bash: line 1: ./vcpkg: Is a directory


In [1]:
!pip install -U transformers datasets evaluate trl accelerate peft sentencepiece
!pip install -i https://pypi.org/simple/ bitsandbytes

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple/


In [2]:
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
import os

import nltk
from nltk.tokenize import sent_tokenize
import sentencepiece
import evaluate
from random import randrange

from transformers import DataCollatorForSeq2Seq, TrainingArguments, Trainer, pipeline, BitsAndBytesConfig
from transformers import LlamaTokenizer, LlamaForCausalLM
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from huggingface_hub import HfFolder
from trl import SFTTrainer

/home/arushi/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [4]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"

In [5]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [6]:
class Llama:
    def __init__(self, dataset_id, model_id):
        self.dataset_id = dataset_id
        self.model_id = model_id
        self.dataset = None
        self.tokenizer = None
        self.model = None
        self.data_collator = None
        self.training_args = None
        self.training_args = None
        self.trainer = None
        self.summarizer = None
        self.peft_config = None
        self.max_source_length = None
        self.max_target_length = None
        self.initialize()
        self.train_model()

    def initialize(self):
            # Load dataset from the hub
            self.dataset = load_dataset(self.dataset_id)

            self.dataset = self.dataset['train']

            # Load tokenizer
            self.tokenizer = LlamaTokenizer.from_pretrained(self.model_id)
            self.model = LlamaForCausalLM.from_pretrained(
                self.model_id,
                load_in_8bit=True,
                use_cache=False,
                use_flash_attention_2=False,
                device_map='auto',
                )

            self.tokenizer.pad_token = self.tokenizer.eos_token
            self.tokenizer.padding_side = "right"

            # LoRA config based on QLoRA paper
            self.peft_config = LoraConfig(
                    lora_alpha=16,
                    lora_dropout=0.1,
                    r=64,
                    bias="none",
                    task_type="CAUSAL_LM",
            )

            self.model = prepare_model_for_kbit_training(self.model)


    def train_model(self):

      def format_prompt(sample):
        return f"""### Instruction:
        Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

        ### Input:
        {sample['response']}

        ### Response:
        {sample['instruction']}
        """

      try:
        # Additional setup for training
        # model = AutoModelForSeq2SeqLM.from_pretrained(self.model_id)
        repository_id = f"{self.model_id.split('/')[1]}-{self.dataset_id}"
        # Define training args
        training_args = TrainingArguments(
            output_dir="llama-7-int4-dolly",
            num_train_epochs=3,
            per_device_train_batch_size=6,
            gradient_accumulation_steps=2,
            gradient_checkpointing=True,
            optim="paged_adamw_32bit",
            logging_steps=10,
            save_strategy="epoch",
            learning_rate=2e-4,
            bf16=False,
            fp16=False,
            tf32=False,
            max_grad_norm=0.3,
            warmup_ratio=0.03,
            lr_scheduler_type="constant",
            warmup_steps=0,
            save_total_limit=3,
            disable_tqdm=False,  # disable tqdm since with packing values are in correct
            )

        model = get_peft_model(self.model, self.peft_config)

        # Create Trainer instance
        trainer = SFTTrainer(
            model=model,
            train_dataset = self.dataset,
            peft_config = self.peft_config,
            formatting_func = format_prompt,
            tokenizer=self.tokenizer,
            dataset_text_field="text",
            max_seq_length=2048,
            args=training_args,
            packing=True,
            )

        if torch.cuda.is_available():
            generator = torch.Generator('cuda').manual_seed(seed)
        else:
            generator = torch.Generator().manual_seed(seed)

        # Start training
        trainer.train()


      except Exception as e:
        print(f"Error during training: {str(e)}")


In [7]:
obj = Llama(dataset_id = 'databricks/databricks-dolly-15k', model_id = 'openlm-research/open_llama_3b_v2')

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/home/arushi/.local/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead o

ValueError: 
                    Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the
                    quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules
                    in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to
                    `from_pretrained`. Check
                    https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
                    for more details.
                    

In [9]:
import torch

# Check available devices
if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print(f"Found {device_count} CUDA device(s)")
else:
    print("CUDA is not available")

# # Load your model
# model = ()

# Check model's device
model_device = next(model.parameters()).device
print(f"Model is on device: {model_device}")

Found 1 CUDA device(s)


NameError: name 'model' is not defined